In [185]:
from __future__ import print_function
import json
from pprint import pprint
import numpy as np

import keras
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from keras import optimizers
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import Lambda, Activation,recurrent, Bidirectional, Dense, Flatten, Conv1D, Dropout, LSTM, GRU, concatenate, multiply, add, Reshape, MaxPooling1D, BatchNormalization
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.engine.topology import Layer

In [2]:
def get_dictionary(vocab):
    d = dict()
    with open(vocab) as f:
        lines = f.readlines()
        for l in lines:
            values = l.strip().split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            d[word] = coefs
    return d

In [3]:
d = get_dictionary("glove/glove.6B.300d.txt")

In [141]:
with open('train.json') as data_file:    
    data = json.load(data_file)

In [164]:
ids = []
titles = dict()
contexts = dict()
questions = dict()
answers_text = dict()
answers_start = dict()
for i in range(len(data)):
    paragraphs = data[i]["paragraphs"]
    title = data[i]["title"]
    for j in range(len(paragraphs)):
        context = paragraphs[j]["context"]
        qas = paragraphs[j]["qas"]
        for k in range(len(qas)):
            id_ = qas[k]["id"]
            answer = qas[k]["answer"]
            question = qas[k]["question"]
            ids.append(id_)
            titles[id_] = title
            contexts[id_] = context
            answers_start[id_] = answer["answer_start"]
            answers_text[id_] = answer["text"]
            questions[id_] = question


In [166]:
max_para = 600
max_q = 50
dimension = 300
train_len = len(ids)

In [167]:
paras = np.zeros((train_len, max_para, dimension))
qns = np.zeros((train_len, max_q, dimension))
exact_match = np.zeros((train_len, max_para, 3))
lmtzr = WordNetLemmatizer()

In [168]:
for i in range(train_len):
    if i % 1000 == 0:
        print(i)
    words = word_tokenize(contexts[ids[i]])
    qs = word_tokenize(questions[ids[i]])
    qs_1 = [w.lower() for w in qs]
    qs_2 = [lmtzr.lemmatize(w) for w in qs_1]
    for j in range(min(max_para - 1, len(words))):
        if words[j].lower() in d:
            paras[i][j] = d[words[j].lower()]
        if words[j] in qs:
            exact_match[i][j][0] = 1
        if words[j].lower() in qs_1:
            exact_match[i][j][1] = 1
        if lmtzr.lemmatize(words[j].lower()) in qs_2:
            exact_match[i][j][2] = 1
    for j in range(min(max_q - 1, len(qs))):
        if qs[j].lower() in d:
            qns[i][j] = d[qs[j].lower()]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000


In [169]:
def create_one_hot_answer(para, answer, answer_start, option, max_length):
    if option == "s":
        from_begin = para[0:answer_start]
    else:
        from_begin = para[0:answer_start+len(answer)]
    l = len(word_tokenize(from_begin))
    one_hot = np.zeros(max_length)
    if option == "s":
        one_hot[min(max_para-1,l)] = 1
    else:
        one_hot[min(max_para-1,l-1)] = 1
    return one_hot
    

In [170]:
ans_starts = [create_one_hot_answer(contexts[ids[i]], answers_text[ids[i]], answers_start[ids[i]], "s", max_para) for i in range(train_len)]
ans_ends = [create_one_hot_answer(contexts[ids[i]], answers_text[ids[i]], answers_start[ids[i]], "e", max_para) for i in range(train_len)]

In [171]:
ans_s = np.array(ans_starts)
ans_e = np.array(ans_ends)

In [172]:
class MyLayer(Layer):

    def __init__(self, output_dim, name, **kwargs):
        self.output_dim = output_dim
        self.name = name
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(name=self.name, 
                                      shape=(input_shape[2], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        return K.dot(x, self.W)

    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[1], self.output_dim)

In [173]:
P = layers.Input(shape=(max_para,dimension), dtype='float32')
Q = layers.Input(shape=(max_q,dimension), dtype='float32')
P_exact_match = layers.Input(shape=(max_para,3), dtype='float32')

In [174]:
alpha = MyLayer(128,"alpha")
beta = MyLayer(1,"beta")
gamma_s = MyLayer(128, "gamma_s")
gamma_e = MyLayer(128, "gamma_e")
hidden_unit = 64

In [175]:
aligned_P = layers.Activation("relu")(alpha(P))
aligned_Q = layers.Activation("relu")(alpha(Q))
coatt_aligned_PQ = layers.dot([aligned_Q, aligned_P], axes=2)
coatt_aligned_PQ = Reshape((max_para * max_q,))(coatt_aligned_PQ)
coatt_aligned_PQ = layers.Activation("softmax")(coatt_aligned_PQ)
coatt_aligned_PQ = Reshape((max_q, max_para,))(coatt_aligned_PQ)
coatt_aligned_PQ = layers.dot([coatt_aligned_PQ,Q],axes=1)

In [176]:
final_P = layers.concatenate([P, P_exact_match, coatt_aligned_PQ], axis=2)
final_P = Dropout(.3)(final_P)
final_P = Bidirectional(LSTM(hidden_unit,return_sequences=True))(final_P)
final_P = Dropout(.3)(final_P)
final_P = Bidirectional(LSTM(hidden_unit,return_sequences=True))(final_P)
final_P = Dropout(.3)(final_P)
final_P = Bidirectional(LSTM(hidden_unit,return_sequences=True))(final_P)
final_P = Dropout(.3)(final_P)

In [177]:
final_Q = Dropout(.3)(Q)
final_Q = Bidirectional(LSTM(hidden_unit,return_sequences=True))(final_Q)
final_Q = Dropout(.3)(final_Q)
final_Q = Bidirectional(LSTM(hidden_unit,return_sequences=True))(final_Q)
final_Q = Dropout(.3)(final_Q)
final_Q = Bidirectional(LSTM(hidden_unit,return_sequences=True))(final_Q)
final_Q = Dropout(.3)(final_Q)
w = beta(final_Q)
final_Q = layers.dot([w,final_Q],axes=1)

In [178]:
start = gamma_s(final_P)
start = layers.dot([start, final_Q], axes=2)
start = Reshape((max_para,))(start)
start = layers.Activation("softmax")(start)

In [179]:
end = gamma_e(final_P)
end = layers.dot([end, final_Q], axes=2)
end = Reshape((max_para,))(end)
end = layers.Activation("softmax")(end)

In [187]:
model = Model([P, Q, P_exact_match],[start, end])
model.load_weights('drqa.h5')

In [192]:
model.compile(optimizer="adamax",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [193]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 600, 300)     0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 50, 300)      0                                            
__________________________________________________________________________________________________
my_layer_16 (MyLayer)           multiple             38400       input_16[0][0]                   
                                                                 input_17[0][0]                   
__________________________________________________________________________________________________
activation_27 (Activation)      (None, 50, 128)      0           my_layer_16[1][0]                
__________

In [ ]:
print('Training')
for i in range(10):
    print("*********************--", i, "--*********************")
    model.fit([paras, qns, exact_match], [ans_s, ans_e],
              batch_size=32, epochs=5, validation_split=0.1)
    model.save('drqa.h5')

Training
*********************-- 0 --*********************
Train on 55241 samples, validate on 6138 samples
Epoch 1/5
